# Building a methylation-informed regulatory network
Daniel C Morgan<sup>1</sup>

<sup>1</sup>Channing Division of Network Medicine, Harvard Medical School, Boston, MA.

## Introduction

Inexpensive, high quality methylation platforms (illumina 450k/850k arrays) have shifted the prospect of integrating such omics into more traditional resources for the inference of gene regulatory networks (GRN). The recent literature has expanded in every direction to take advantage of this new, ubiquitous data source, and as such a wide range of approaches have been proposed. Recent publications range from potential use cases where it makes the most sense to account for DNA methylation in regulatory framework to methodological approaches for doing just that without regard for the research question. This book chapter is organized based upon this continuum, starting with use cases most appropriate for taking advantage of DNA methylation information towards more recent approaches of methodologies and frameworks for appropriately and accurately doing so. Lastly, we shall work through a modern approach for both, namely a meGRN framework and use case example, to see the utility methylation can lend to an investigation.

## Loading libraries

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import scipy
import pandas as pd
# from pybedtools import BedTool ##super limited capabilities

## Motivation

The matter involves finding regions of known TF binding which also have methylation events proximal to these locations. For this, one can rely on WGBS chromosome location or array data which illumina provides cg names for, which map to similar locations.


## Outline:
- Brief literature review
- Data formats -- what is required, depending on availability
  - Chr start stop
  - Cg annotation (illumina)
  - Other → some method for relating genome location to binding
- Pipeline:
  0. FIMO scan & open-access data
  1. (py)bedtools
  2. integrating
    1. Integrating into netzoo bipartite
    1. Integrating into other GRN framework
      1. Smaller GRN models (TF-gene subsets) could just merge with methyl-motif and reduce their estimates where overlap / downweight (GRN estimate x 1- avg meth ratio)
  3. Benchmarking against ChIP-seq

## Building PWM Motif

In [ ]:
# !wget https://granddb.s3.amazonaws.com/optPANDA/motifs/regMatPval1e3.csv
# data=pd.read_csv('regMatPval1e3.csv')
# mdata=pd.melt(data,id_vars=['Unnamed: 0'])
# mdata=mdata.rename(columns={'Unnamed: 0':'TF','variable':'gene','value':'val'})
# mdata.to_csv('regPval1e3.bed',sep='\t',header=False)
# !rm regMatPval1e3.csv

In [17]:
9500113165461078524477

In [18]:
##TSS

#https://raw.githubusercontent.com/Shicheng-Guo/AnnotationDatabase/master/hg38/refGene.hg38.TSS.bed

## Gathering data

In [4]:
TF > [TSS > WGBS] > ChIP

'/Users/redmo/MIMB'

In [1]:
## A549 WGBS
# !wget https://www.encodeproject.org/files/ENCFF005TID/@@download/ENCFF005TID.bed.gz
# !gunzip ENCFF005TID.bed.gz

# ## PPI
# !wget https://granddb.s3.amazonaws.com/gpuPANDA/ppi2015_freezeCellLine.txt

## remap ChIP (filter to A549)
# !wget http://remap.univ-amu.fr/storage/remap2020/hg38/MACS2/remap2020_all_macs2_hg38_v1_0.bed.gz
# !gunzip remap2020_all_macs2_hg38_v1_0.bed.gz

# read conversion file
# !unzip hg38_Tss_coordinates.csv.zip
# geneCorr = pd.read_csv('hg38_Tss_coordinates.csv',sep='\t')
# geneCorr[['chrom','txStart','txEnd','name2']].to_csv('hg38_gene_coord.txt',sep='\t',header=False,index=False)


In [6]:
##install bedtools
# https://bedtools.readthedocs.io/en/latest/content/installation.html

In [20]:
! bedtools intersect -wa -wb -a ENCFF005TID.bed -b hg38_gene_coord.txt |cut -f1,2,3,10,11,15 |uniq> A.txt

In [21]:
data=pd.read_csv('M5568_1.02.txt',sep='\t')

In [22]:
data[['sequence_name','start','stop','motif_alt_id']].to_csv('generic_motif.txt',sep='\t',index=False,header=False)


In [24]:
! bedtools intersect -wa -wb -a generic_motif.txt -b A.txt > B.txt

In [5]:
! wc -l A.txt B.txt generic_motif.txt

 37526328 A.txt
   68840 B.txt
  162636 generic_motif.txt
 37757804 total


In [8]:
! bedtools slop -i generic_motif.txt -g human.hg38.genome -r 250 -l 250 > generic_motif_250.txt


In [9]:
! bedtools intersect -wa -wb -a generic_motif_250.txt -b A.txt > B250.txt

In [10]:
! wc -l A.txt B250.txt generic_motif_250.txt

 37526328 A.txt
  814273 B250.txt
  162636 generic_motif_250.txt
 38503237 total


In [7]:
! bedtools intersect -wa -wb -a Motif_BS_Ch.txt -b hg38_tss_coord.txt > Motif_BS_Chmax_refseq.txt

Error: Unable to open file Motif_BS_Ch.txt. Exiting.


In [5]:
data=pd.read_csv(uniqMotif_BS_Chmax_refseq.txt,sep='\t')

wc: uniqMotif_BS_Chmax_refseq.txt: open: No such file or directory


In [ ]:
methyl_motif['TF']=methyl_motif['TF'].str.replace(r"\(.*\)","")
m_motif=methyl_motif[['TF','gene','weight','W1','ChIPTF']]
# um_motif=m_motif.drop_duplicates()
min_um_motif=motif_file.merge(m_motif,on=['TF','gene'])
min_um_motif['W1']=np.round(min_um_motif['W1'],decimals=3)
mm=min_um_motif.groupby(['TF','gene']).max()
mmm=mm.reset_index()
mmm[['TF','gene','W1']].to_csv('min_0max_methyl_motif.txt',sep='\t',header=False,index=False)
mmm[['TF','gene','weight_y']].to_csv('min_0max_pwm_motif.txt',sep='\t',header=False,index=False)
# mmm[['TF','gene','ChIPTF']].to_csv('min_0max_ChIPTF_motif.txt',sep='\t',header=False,index=False)
mmm.weight_y=1
mmm[['TF','gene','weight_y']].to_csv('min_0one_motif.txt',sep='\t',header=False,index=False)


nn=min_um_motif.groupby(['TF','gene']).mean()
nnn=nn.reset_index()
nnn[['TF','gene','W1']].to_csv('min_0mean_methyl_motif.txt',sep='\t',header=False,index=False)
nnn[['TF','gene','weight_y']].to_csv('min_0mean_pwm_motif.txt',sep='\t',header=False,index=False)


In [8]:
def test_panda(motif,size):
    panda_objC=netZooPy.panda.Panda(expression_file=None,#'drive/MyDrive/Colab Notebooks/milipede_bench/Hugo_exp1_lcl.txt',
                          motif_file='drive/MyDrive/Colab Notebooks/milipede_bench/'+size+'/'+motif,
                          ppi_file='ppi2015_freezeCellLine.txt',
                          computing='cpu',modeProcess='intersection',save_memory=False,save_tmp=False,
                          precision='single',keep_expression_matrix=False)
    panda_objC.export_panda_results.to_csv('bench/'+size+'/output_'+size+'/'+motif,sep='\t',header=True,index=False)
    return panda_objC.export_panda_results

In [ ]:
#mean no buffer
test_panda(min_0mean_methyl_motif,0bp_bench)
test_panda(min_0mean_pwm_motif,0bp_bench)
test_panda(min_0max_one_motif,0bp_bench)

##max no buffer
test_panda(min_0max_methyl_motif,0bp_bench)
test_panda(min_0max_pwm_motif,0bp_bench)
# test_panda(min_0max_one_motif,0bp_bench) not needed

##mean 100bp buffer
test_panda(min_100mean_methyl_motif,100bp_bench)
test_panda(min_100mean_pwm_motif,100bp_bench)
test_panda(min_100mean_one_motif,100bp_bench)

##max 100bp buffer
test_panda(min_100max_methyl_motif,100bp_bench)
test_panda(min_100max_pwm_motif,100bp_bench)
test_panda(min_100max_one_motif,100bp_bench)


In [6]:
267+231

498

In [7]:
267+377

644